In [6]:
import cv2
import mediapipe as mp
import os
import json
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [4]:
import cv2
import mediapipe as mp
import os
import json
import numpy as np

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# Placeholder for the directory containing the videos
video_path = "/Users/cezar/Desktop/Team Project/AI/shotput_models/stage2/keypoint_videos_augmented"

# Function to calculate angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Second point (vertex)
    c = np.array(c)  # Third point
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.degrees(np.arccos(cosine_angle))
    return angle

# Process videos in the directory
for file in os.listdir(video_path):
    if file.endswith(".mp4"):
        video_file_path = os.path.join(video_path, file)
        cap = cv2.VideoCapture(video_file_path)

        keypoints_data = []  # Store keypoints for this video

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Convert frame to RGB
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            result = pose.process(frame_rgb)

            if result.pose_landmarks:
                landmarks = result.pose_landmarks.landmark

                # Extract relevant keypoints for the trailing leg
                hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP].x,
                       landmarks[mp_pose.PoseLandmark.LEFT_HIP].y]
                knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_KNEE].y]
                ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].y]

                # Calculate angle and distance
                angle = calculate_angle(hip, knee, ankle)

                # Store data for this frame
                keypoints_data.append({
                    "frame": cap.get(cv2.CAP_PROP_POS_FRAMES),
                    "angle": angle,
                    "hip": hip,
                    "knee": knee,
                    "ankle": ankle
                })

        # Release the video
        cap.release()

        # Save keypoints to a JSON file
        json_filename = os.path.splitext(file)[0] + "_hop_phase_keypoints.json"
        json_path = os.path.join(video_path, json_filename)
        with open(json_path, "w") as json_file:
            json.dump(keypoints_data, json_file, indent=4)

print("Keypoints extraction for hop phase complete!")

I0000 00:00:1736608465.231470 7694878 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M2 Pro
W0000 00:00:1736608465.299488 7702775 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1736608465.313189 7702775 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Keypoints extraction for hop phase complete!


In [3]:
def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Second point (vertex)
    c = np.array(c)  # Third point
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    return np.degrees(np.arccos(cosine_angle))

def calculate_distance(a, b):
    a = np.array(a)
    b = np.array(b)
    return np.linalg.norm(a - b)

In [3]:
import cv2
import os
import numpy as np

# Placeholder for input and output directories
input_video_path = "/Users/cezar/Desktop/Team Project/AI/shotput_models/stage2/keypoint_videos"
augmented_video_path = "/Users/cezar/Desktop/Team Project/AI/shotput_models/stage2/keypoint_videos_augmented"
os.makedirs(augmented_video_path, exist_ok=True)

# Define augmentation functions
def augment_video(video_path, output_path, augmentations):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

    # Create an augmented video writer
    out = cv2.VideoWriter(output_path, fourcc, fps, frame_size)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Apply augmentations
        for aug in augmentations:
            augmented_frame = aug(frame)
            out.write(augmented_frame)

    cap.release()
    out.release()

# Example augmentations
def mirror(frame):
    return cv2.flip(frame, 1)

def add_noise(frame):
    noise = np.random.normal(0, 10, frame.shape).astype(np.uint8)
    return cv2.add(frame, noise)

def rotate(frame, angle=5):
    height, width = frame.shape[:2]
    matrix = cv2.getRotationMatrix2D((width // 2, height // 2), angle, 1)
    return cv2.warpAffine(frame, matrix, (width, height))

# Apply augmentations to all videos
for file in os.listdir(input_video_path):
    if file.endswith(".mp4"):
        video_file_path = os.path.join(input_video_path, file)
        base_name = os.path.splitext(file)[0]

        # Augment and save
        augment_video(video_file_path, os.path.join(augmented_video_path, f"{base_name}_mirrored.mp4"), [mirror])
        augment_video(video_file_path, os.path.join(augmented_video_path, f"{base_name}_noisy.mp4"), [add_noise])
        augment_video(video_file_path, os.path.join(augmented_video_path, f"{base_name}_rotated.mp4"), [lambda frame: rotate(frame, 5)])

In [7]:
import json

features = []
labels = []

# Placeholder for the JSON directory
keypoints_directory = "/Users/cezar/Desktop/Team Project/AI/shotput_models/stage2/keypoint_videos_augmented"

# Load JSON files and extract features
for file in os.listdir(keypoints_directory):
    if file.endswith(".json"):
        with open(os.path.join(keypoints_directory, file), "r") as f:
            data = json.load(f)

        angles = []
        distances = []

        for frame in data:
            hip = frame["hip"]
            knee = frame["knee"]
            ankle = frame["ankle"]

            # Calculate angle and distance
            angle = calculate_angle(hip, knee, ankle)
            distance = calculate_distance(hip, knee)

            angles.append(angle)
            distances.append(distance)

        # Combine features
        feature_sequence = np.stack([angles, distances], axis=1)
        features.append(feature_sequence)

        # Extract label from filename (e.g., 0, 0.5, 1)
        label = float(file.split("_")[0])  # Adjust this based on your naming convention
        labels.append(label)

# Pad sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_len = max(len(seq) for seq in features)
features = pad_sequences(features, maxlen=max_len, padding="post", dtype="float32")
labels = np.array(labels)


In [8]:
# Train-test split
X_train, X_val, y_train, y_val = train_test_split(
    features, labels, test_size=0.15, random_state=42  # Adjust split for 17 videos
)

# Reshape input for LSTM
X_train = X_train[..., np.newaxis]  # Add channel dimension for LSTM
X_val = X_val[..., np.newaxis]

print(f"Training samples: {len(X_train)}, Validation samples: {len(X_val)}")


Training samples: 43, Validation samples: 8


In [35]:
# Define the LSTM model
model = Sequential([
    LSTM(64, activation='tanh', return_sequences=False, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.15),
    Dense(32, activation='relu'),
    Dense(1)  # Output: Regression score (0, 0.5, 1)
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='mse', metrics=['mae'])

# Model summary
print(model.summary())

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_5 (LSTM)                   │ (None, 64)             │        17,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,265 (75.25 KB)

 Trainable params: 19,265 (75.25 KB)

 Non-trainable params: 0 (0.00 B)

None


In [36]:
# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=800,  # Adjust epochs based on convergence
    batch_size=8,  # Smaller batch size for small dataset
    verbose=1
)

Epoch 1/800
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.7324 - mae: 0.7276 - val_loss: 0.9907 - val_mae: 0.9953
Epoch 2/800
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.7254 - mae: 0.7317 - val_loss: 0.9789 - val_mae: 0.9894
Epoch 3/800
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.8435 - mae: 0.8404 - val_loss: 0.9665 - val_mae: 0.9831
Epoch 4/800
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.7547 - mae: 0.7758 - val_loss: 0.9533 - val_mae: 0.9764
Epoch 5/800
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.7746 - mae: 0.8022 - val_loss: 0.9390 - val_mae: 0.9690
Epoch 6/800
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.8031 - mae: 0.8253 - val_loss: 0.9230 - val_mae: 0.9607
Epoch 7/800
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.7292 - mae: 0.7833 - val_loss: 0.9049 - val_mae: 0.9513
Epoch 8/800
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.6960 - mae: 0.7407 - val_loss: 0.8856 - val_mae: 0.9410
Epoch 9/800
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.7165 - mae: 0.7800 -

In [37]:
val_loss, val_mae = model.evaluate(X_val, y_val, verbose=0)
print(f"Validation Loss: {val_loss:.4f}, Validation MAE: {val_mae:.4f}")

Validation Loss: 0.1147, Validation MAE: 0.1431


In [38]:
model.save("shotput_stage2.keras")

In [39]:
# Predict scores on validation data
predictions = model.predict(X_val)

# Print predictions vs actual
for i, (pred, actual) in enumerate(zip(predictions, y_val)):
    print(f"Predicted: {pred[0]:.2f}, Actual: {actual:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
Predicted: 1.00, Actual: 1.00
Predicted: 0.96, Actual: 1.00
Predicted: 0.05, Actual: 1.00
Predicted: 0.98, Actual: 1.00
Predicted: 0.92, Actual: 1.00
Predicted: 0.99, Actual: 1.00
Predicted: 0.97, Actual: 1.00
Predicted: 1.01, Actual: 1.00


In [59]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Function to calculate the angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Second point (vertex)
    c = np.array(c)  # Third point
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    return np.degrees(np.arccos(cosine_angle))

# Function to calculate distance between two points
def calculate_distance(a, b):
    a = np.array(a)
    b = np.array(b)
    return np.linalg.norm(a - b)

# Load the trained model
model_path = "/Users/cezar/Desktop/Team Project/AI/shotput_models/stage2/shotput_stage2.keras"
model = load_model(model_path)

# Path to the new video
new_video_path = "/Users/cezar/Desktop/Team Project/AI/shotput_models/stage2/keypoint_videos/0_user13.mp4"  # Replace with the actual video path

# Extract features from the new video
angles = []
distances = []

cap = cv2.VideoCapture(new_video_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = pose.process(frame_rgb)

    if result.pose_landmarks:
        landmarks = result.pose_landmarks.landmark

        # Extract relevant keypoints for the trailing leg (right leg in this case)
        right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP].x,
                     landmarks[mp_pose.PoseLandmark.RIGHT_HIP].y]
        right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE].x,
                      landmarks[mp_pose.PoseLandmark.RIGHT_KNEE].y]
        right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE].y]

        # Calculate angle and distance
        angle = calculate_angle(right_hip, right_knee, right_ankle)
        distance = calculate_distance(right_hip, right_knee)

        angles.append(angle)
        distances.append(distance)

cap.release()

# Combine features
features = np.stack([angles, distances], axis=1)

# Pad the sequence to match the model's input shape
features = pad_sequences([features], maxlen=model.input_shape[1], padding='post', dtype='float32')
features = features[..., np.newaxis]  # Add channel dimension for single feature

# Predict the score for the new video
predicted_score = model.predict(features)
print(f"Predicted Score: {predicted_score[0][0]:.2f}")


I0000 00:00:1736688063.098269 7938281 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M2 Pro
W0000 00:00:1736688063.198559 8365498 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1736688063.212584 8365502 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
Predicted Score: 0.22


In [55]:
def classify_score(prediction):
    """Classify the prediction into 0, 0.5, or 1 based on thresholds."""
    if prediction >= 0.85:
        return 1.0
    elif prediction >= 0.70:
        return 0.5
    else:
        return 0.0

In [56]:
# Predict the score for the new video
predicted_score = model.predict(features)
print(f"Predicted Score (Regression): {predicted_score[0][0]:.2f}")

# Classify the score
classified_score = classify_score(predicted_score[0][0])
print(f"Classified Score: {classified_score}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Predicted Score (Regression): 0.73
Classified Score: 0.5
